In [1]:
from tqdm import tqdm
import numpy as np
import pandas as pd
from lxml import html
import requests
import re
import csv
from elasticsearch import Elasticsearch
from elasticsearch import helpers
indexingTables = Elasticsearch()

In [2]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
stop_words = set(stopwords.words('english')) 

In [3]:
def removeStopwords(example_sent):
    
    word_tokens = word_tokenize(example_sent) 
  
    filtered_sentence = [w for w in word_tokens if not w in stop_words] 
  
    filtered_sentence = [] 
  
    for w in word_tokens: 
        if w not in stop_words: 
            filtered_sentence.append(w)
    
    formattedText = ""

    for word in filtered_sentence:
        
        if (len(word)>2):
            
            formattedText = formattedText + " " +word
    
    
    formattedText = formattedText.lstrip()
    
    formattedText = formattedText.rstrip()
        
    return formattedText

In [4]:
articles = pd.read_csv('../articlesDataset/articlesPlusTitle', delimiter=',', header=None)
tables = pd.read_csv('../articlesDataset/cleanDataTables', delimiter=',', header=None)

In [5]:
dataArticles = articles.iloc[:,:].values
dataTables = tables.iloc[:,:].values

In [ ]:
distinctArticlesIDs = []
distinctArticles = []

for articles in tqdm(dataArticles):
    
    articlePgID = articles[0]
    
    articleTitle = articles[1]
    
    articleText = articles[2]
    
    if articlePgID not in distinctArticlesIDs:
        
        distinctArticlesIDs.append(articlePgID)
        
        distinctArticles.append([articlePgID,articleTitle, articleText])

In [ ]:
with open('distinctArticles', 'w') as myfile:
        
    wr = csv.writer(myfile, quoting=csv.QUOTE_ALL)

    for articles in tqdm(distinctArticles):
    
        articlePgID = articles[0]
        articleTitle = str(articles[1])
        
        row = [articlePgID,articleTitle]
        
        wr.writerow(row)

In [4]:
distinctArticles = pd.read_csv('../articlesDataset/distinctArticles', delimiter=',', header=None)

In [5]:
len(distinctArticles)

66234

In [6]:
distinctArticles = distinctArticles.dropna(how='any',axis=0)

In [7]:
len(distinctArticles)

66187

In [8]:
dataDistinctArticles = distinctArticles.iloc[:,:].values

In [9]:
def searchIndexingMatch(articleID):
    
    result= indexingTables.search(
        index="tables", 
        body = {
        "_source": ["tablePgTitle"],
        "from" : 0,
        "size" : 1,
        "query": {
            "match" : {
                "tablePgID" : {
                    "query" : articleID
                    }
                }
        }
    })
    
    return result

In [10]:
def searchIndexingNoMatch(articleTitle):
    
    result= indexingTables.search(
        index="tables", 
        body = {
        "_source": ["tablePgID","tablePgTitle"],
        "from" : 0,
        "size" : 20,
        "query": {
            "multi_match":{
              "type": "most_fields",
              "query":    articleTitle, 
              "fields": ["tablePgTitle"] 
            }
        }
    })
    
    return result

In [11]:
finalDataset = []

for i in tqdm(range(len(dataDistinctArticles))):
    
    articleID = dataDistinctArticles[i][0]
    articleTitle = dataDistinctArticles[i][1]
    
    #getting the table title match
    result1 = searchIndexingMatch(articleID)
    for hit in result1['hits']['hits']:
    
        tableTitleMatch = removeStopwords(hit['_source']['tablePgTitle'])
            
    tableTitleMatch = tableTitleMatch
    
    
    #getting the table title non-match
    result2 = searchIndexingNoMatch(articleTitle)
    for hit in result2['hits']['hits']:
        
        tableID = hit['_source']['tablePgID']
        tableTitleNonMatch = removeStopwords(hit['_source']['tablePgTitle'])
        
        if articleID != tableID: 
            
            break
    
    tableTitleNoMatch = tableTitleNonMatch
    
    finalDataset.append([articleID,articleTitle,tableTitleMatch,1])
    finalDataset.append([articleID,articleTitle,tableTitleNoMatch,0])

100%|██████████| 66187/66187 [20:10<00:00, 54.69it/s]


In [12]:
len(finalDataset)

132374

In [16]:
with open('trainDatasetTitle', 'w') as myfile:
        
    wr = csv.writer(myfile, quoting=csv.QUOTE_ALL)

    for i in range(19855,len(finalDataset)):
    
        articlePgID = finalDataset[i][0]
        articleTitle = str(finalDataset[i][1])
        tableTitle = str(finalDataset[i][2])
        match = finalDataset[i][3]
        
        row = [articlePgID,articleTitle,tableTitle,match]
        
        wr.writerow(row)